In [1]:
import jwt
import msal

In [2]:
# DocHuntVue stuff
client_id = '7a0d396a-c9aa-4a65-8bff-e906fef7b0c7'
client_credential = '24597952-3fe5-42be-ab7b-6308311158dd'
authority = (
    'https://clunacy.b2clogin.com/clunacy.onmicrosoft.com/'
    'B2C_1_dochuntsignupsignin'
)

In [5]:
client = msal.ConfidentialClientApplication(
    client_id,
    client_credential=client_credential,
    authority=authority,
)

In [11]:
response = client.acquire_token_for_client(['results.read.all'])

In [12]:
response

{'error': 'unsupported_grant_type',
 'error_description': 'AADB2C90086: The supplied grant_type [client_credentials] is not supported.\r\nCorrelation ID: 751587b6-47ac-4e1e-8311-006f0dd0b9f0\r\nTimestamp: 2021-02-02 12:36:26Z\r\n'}